## Постановка задачи

### Проблема:
Booking — американский поисковик-агрегатор, позволяющий бронировать места в ресторанах, отелях, арендовать авто и т.д. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

### Задача:
Создать модель, которая должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных. Изученные нами навыки разведывательного анализа помогут улучшить модель. Как? Узнаем дальше.

In [201]:
import pandas as pd
import category_encoders as ce

## Информация о датасете

Первоначальная версия датасета содержит 17 полей со следующей информацией:

* hotel_address — адрес отеля;
* review_date — дата, когда рецензент разместил соответствующий отзыв;
* average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
* hotel_name — название отеля;
* reviewer_nationality — страна рецензента;
* negative_review — отрицательный отзыв, который рецензент дал отелю;
* review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
* positive_review — положительный отзыв, который рецензент дал отелю;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.

Размер таблицы: (386803;17)
Пропуски в данных: 'lat', 'lng'

In [202]:
hotels = pd.read_csv('hotels.csv')
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [203]:
hotels = pd.read_csv('hotels.csv')
hotels.head(5)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


## Подготовка данных к обучению модели

In [204]:
# Преобразуем информацию о дате в формат datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'],yearfirst=True)

Признак 'tags' содержит информацию о тэгах пользователей, оставленных в каждом коментариии к отелю, инфромация в ячейках представлена в виде строки. Нам необходимо получить из строки список тэгов. 

In [205]:
# Функция принимает строку с тэгами и возвращает список тэгов
def get_tags_list(tags_str):

    tags_list = tags_str[1:-1] # удаляем скобки вначале и в конце строки
    tags_list = tags_list.replace("' ","") # удаляем кавычки вначале тэга
    tags_list = tags_list.replace(" '","") # удаляем кавычки вконце тэга
    tags_list = tags_list.split(', ') # разделяем строку на отдельные тэги
    return tags_list

hotels['tags'] = hotels['tags'].apply(get_tags_list)

Попробуем выяснить какие тэги пользователи оставляли чаще, а какие реже.

In [209]:
# Функция принимает на вход список тэгов
# и возвращает словарь тэгов, где значения - количество их повтров в ячейках таблицы

uniq_tags_dict = dict() # задаем словарь, в который будем добавлять тэги

def tags_count(tags_list):
    for elem in tags_list:
        if elem in uniq_tags_dict.keys():
            uniq_tags_dict[elem] = int(uniq_tags_dict.get(elem)) + 1 
        else:
            uniq_tags_dict[elem] = 1
    return(uniq_tags_dict)
    
hotels['tags'].apply(tags_count)

# Количество уникальных тэгов в таблице
print("Количество уникальных тэгов в таблице:",len(uniq_tags_dict))

# Отсоритируем словарь с тэгами по значению в порядке убывания (сохранение порядка элементов словаря
# не гарантируется в версиях Python старше 3.7) 

sorted_values = sorted(uniq_tags_dict.values(),reverse=True) # Создадим отсортированный список со значениями 
sorted_dict = {} # пустой словарь, куда будем добавлять отсортированные ключи со значениями

for i in sorted_values:
    for k in uniq_tags_dict.keys():
        if uniq_tags_dict[k] == i:
            sorted_dict[k] = uniq_tags_dict[k]
            break

print(sorted_dict)

Количество уникальных тэгов в таблице: 340
{'Leisure trip': 313593, 'Submitted from a mobile device': 230778, 'Couple': 189212, 'Stayed 1 night': 145373, 'Stayed 2 nights': 100263, 'Solo traveler': 81235, 'Stayed 3 nights': 72000, 'Business trip': 61989, 'Group': 49088, 'Family with young children': 45836, 'Stayed 4 nights': 35748, 'Double Room': 26386, 'Standard Double Room': 24151, 'Superior Double Room': 23550, 'Family with older children': 19802, 'Deluxe Double Room': 18623, 'Double or Twin Room': 16824, 'Stayed 5 nights': 15611, 'Standard Double or Twin Room': 13062, 'Classic Double Room': 12716, 'Stayed 6 nights': 4939, 'Stayed 7 nights': 3704, 'Stayed 8 nights': 1318, 'Stayed 9 nights': 681, 'Stayed 10 nights': 459, 'Stayed 11 nights': 218, 'Stayed 12 nights': 146, 'Stayed 14 nights': 126, 'Stayed 13 nights': 120, 'Standard Twin Room': 65, 'Stayed 15 nights': 64, '2 rooms': 62, 'Twin Room': 59, 'Superior Double or Twin Room': 56, 'Triple Room': 40, 'Superior Twin Room': 35, 'Cla

Итак, в таблице 2368 уникальных тэга, из них чаще всего используется тэг 'Leisure trip' - 313593 раз, 
реже всего тэг 'King Room with Sea View' - 1 раз. Тэги, которые редко используют пользователи, не несут для нас полезной информации поэтому можно оставить только те тэги, которые чаще встречаются в датасете.

In [207]:
# Составим список топ-20 тэгов
top_tags_list = list(sorted_dict.keys())[:20]

# Функция принимает список всех тэгов в ячейке, и возвращает только те тэги, 
# которые есть в списке топ-20 тэгов
def del_tags(tags_list):
    for elem in tags_list:
        if elem not in top_tags_list:
            tags_list.remove(elem)
    return tags_list
    
hotels['tags'].apply(del_tags)

0         [Leisure trip, Couple, Stayed 2 nights, Submit...
1         [Business trip, Couple, Standard Double Room, ...
2         [Leisure trip, Solo traveler, Stayed 3 nights,...
3             [Leisure trip, Solo traveler, Stayed 1 night]
4         [Business trip, Couple, Standard Double or Twi...
                                ...                        
386798    [Leisure trip, Group, Stayed 2 nights, Submitt...
386799    [Leisure trip, Couple, Standard Double Room, S...
386800       [Business trip, Solo traveler, Stayed 1 night]
386801    [Leisure trip, Solo traveler, Deluxe Double Ro...
386802              [Leisure trip, Couple, Stayed 4 nights]
Name: tags, Length: 386803, dtype: object

## !!! Закодировать тэги

In [208]:
top_tags_list 

['Leisure trip',
 'Submitted from a mobile device',
 'Couple',
 'Stayed 1 night',
 'Stayed 2 nights',
 'Solo traveler',
 'Stayed 3 nights',
 'Business trip',
 'Group',
 'Family with young children',
 'Stayed 4 nights',
 'Double Room',
 'Standard Double Room',
 'Superior Double Room',
 'Family with older children',
 'Deluxe Double Room',
 'Double or Twin Room',
 'Stayed 5 nights',
 'Standard Double or Twin Room',
 'Classic Double Room']